In [9]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

np.random.seed(42)

In [5]:
df = pd.read_csv("../datasets/consolidated_data/oil_data.csv", index_col="DATEPRD")

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, index=df.index, columns=df.columns)

date = df.index
scaled_oil_data = scaled_df.BORE_OIL_VOL.to_numpy()

split_date = 2140

date_train = date[:split_date]
date_test = date[split_date:]
oil_train = scaled_oil_data[:split_date]
oil_test = scaled_oil_data[split_date:]

print(oil_train.shape, oil_test.shape, date_train.shape, date_test.shape)

scaled_df.head()

(2140,) (916,) (2140,) (916,)


,BORE_OIL_VOL
DATEPRD,
2008-02-12,0.030164
2008-02-13,0.198133
2008-02-14,0.331061
2008-02-15,0.276400
2008-02-16,0.323400


In [14]:
def refresh():
    keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)

class ResetStatesCallback(Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

def sequential_window_dataset(series, window_size):
    
    series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=window_size, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[1:]))
    
    return dataset.batch(1).prefetch(1)

# def prepare_dataset(data, window_size):
#     X, Y = np.empty((0,window_size)), np.empty((0))
#     for i in range(len(data)-window_size-1):
#         X = np.vstack([X,data[i:(i + window_size),0]])
#         Y = np.append(Y,data[i + window_size,0])   
#     X = np.reshape(X,(len(X),window_size,1))
#     Y = np.reshape(Y,(len(Y),1))
#     return X, Y

def train_model(train_dataset, validation_dataset, num_units):
    model = Sequential([
                    LSTM(num_units, return_sequences=True, stateful=True, batch_input_shape=[1, None, 1]),
                    LSTM(num_units, return_sequences=True, stateful=True),
                    Dense(1)
            ])
    optimizer = keras.optimizers.SGD(
                    learning_rate=1e-3, 
                    momentum=0.9
                )
    model.compile(
                    loss=keras.losses.Huber(), 
                    optimizer=optimizer, 
                    metrics=["mae"]
                )
    reset_states = ResetStatesCallback()
    model_checkpoint = ModelCheckpoint(
                    "LSTM_oil_checkpoint.h5", 
                    save_best_only=True
                    )
    early_stopping = EarlyStopping(patience=50)
    model.fit(
                    train_dataset, 
                    epochs=500, 
                    verbose=0,
                    validation_data=validation_dataset,
                    callbacks=[
                                early_stopping, 
                                model_checkpoint, 
                                reset_states
                            ]
            )

    return model

def train_evaluate(ga_individual_solution):   
    # Decode GA solution to integer for window_size and num_units
    window_size_bits = BitArray(ga_individual_solution[0:8])
    num_units_bits = BitArray(ga_individual_solution[8:]) 
    window_size = window_size_bits.uint
    num_units = num_units_bits.uint
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_units)
    
    # Return fitness score of 100 if window_size or num_unit is zero
    if window_size == 0 or num_units == 0:
        return 100, 
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    refresh()

    oil_train_set = sequential_window_dataset(oil_train, window_size)
    oil_test_set = sequential_window_dataset(oil_test, window_size)
    
    # Train LSTM model and predict on validation set
    model = train_model(oil_train_set, oil_test_set, num_units)
    
    #load best model
    model = keras.models.load_model("LSTM_oil_checkpoint.h5")
    LSTM_oil_forecast = model.predict(scaled_oil_data[np.newaxis, :, np.newaxis])
    LSTM_oil_forecast = LSTM_oil_forecast[0, split_date - 1:-1, 0]

    # # Calculate the RMSE score as fitness score for GA
    mae = keras.metrics.mean_absolute_error(oil_test, LSTM_oil_forecast).numpy()
    mse = keras.metrics.mean_squared_error(oil_test, LSTM_oil_forecast).numpy()
    rmse = math.sqrt(mse)

    print(f'mae = {mae}, \nmse = {mse}, \nrmse = {rmse}')
    
    return rmse,

In [ ]:
population_size = 4
num_generations = 4
gene_length = 16

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)

In [ ]:
best_individuals = tools.selBest(population,k = 1)
best_window_size = None
best_num_units = None

for bi in best_individuals:
    window_size_bits = BitArray(bi[0:8])
    num_units_bits = BitArray(bi[8:]) 
    best_window_size = window_size_bits.uint
    best_num_units = num_units_bits.uint
    print('\nWindow Size: ', best_window_size, ', Num of Units: ', best_num_units)

In [ ]:
X_train,y_train = prepare_dataset(train_data,best_window_size)
X_test, y_test = prepare_dataset(test_data,best_window_size)

inputs = Input(shape=(best_window_size,1))
x = LSTM(best_num_units, input_shape=(best_window_size,1))(inputs)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs = inputs, outputs = predictions)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)